# Extract data from instagram with Selenium

## Imports

In [1]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver.support.wait import WebDriverWait
from bs4 import BeautifulSoup
import pandas as pd
from datetime import date

In [2]:
#Acessando a pagina pelo chrome webdriver
driver = webdriver.Chrome('chromedriver')
driver.get('https://www.instagram.com/')

In [3]:
#Fazendo login
username = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.CSS_SELECTOR, "input[name='username']")))
password = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.CSS_SELECTOR, "input[name='password']")))

username.clear()
password.clear()

username.send_keys('scrap1234567')
password.send_keys('Fe@nor@12345')

log_in = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.CSS_SELECTOR, "button[type='submit']"))).click()


#Pulando os Pop ups
not_now = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, "//button[contains(text(), 'Agora não')]"))).click()
not_now2 = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, "//button[contains(text(), 'Agora não')]"))).click()

In [4]:
#Função para puxar dados de um determinado perfil
def dados_perfil(nome):
    #Procurando um perfil
    searchbox = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, "//input[@placeholder='Pesquisar']")))
    searchbox.clear()
    keyword = nome
    searchbox.send_keys(keyword)
    searchbox.send_keys(Keys.ENTER)
    user = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.CSS_SELECTOR, "a[class='-qQT3']"))).click()

    #Extraindo nome de usuario
    insta_user = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.CSS_SELECTOR, "h2[class='_7UhW9       fKFbl yUEEX   KV-D4              fDxYl     ']"))).text

    #Extraindo o cabeçalho com posts, seguidores e seguindo
    insta_info = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.CSS_SELECTOR, "ul[class='k9GMp ']"))).get_attribute('innerHTML')

    soup = BeautifulSoup(insta_info, 'html.parser')

    posts = soup.findAll('span', {'class':'g47SY'})[0].text
    posts = int(posts.replace('.',''))

    followers = soup.findAll('span', {'class':'g47SY'})[1].attrs['title']
    followers = int(followers.replace('.',''))

    following = soup.findAll('span', {'class':'g47SY'})[2].text
    following = int(following.replace('.',''))

    jogador = {
              'posts':posts,
              'seguidores':followers,
              'seguindo':following}
    
    #Voltando à home do instagram
    back_home = driver.get('https://www.instagram.com/')
    
    return jogador

#Função que lê o arquivo com todos os nomes de jogadores e puxa os dados do dia 
def coleta_dados(usuarios):
    file = open(usuarios).read().split()
    hoje = date.today().strftime('%d-%m-%Y')
    for usuario in file:
        perfil = dados_perfil(usuario)
        
        to_add = perfil
        to_add['data'] = hoje


        df_jogador = pd.read_csv('dados/' + usuario + '.csv')
        df_jogador = df_jogador.append(to_add, ignore_index = True)
        df_jogador.to_csv('dados/' + usuario + '.csv', sep = ',', index = False)
        

In [5]:
file = open('jogadores_insta')
usuarios = file.read().split()
for usuario in usuarios:
    pd.DataFrame(columns = ['data', 'seguidores', 'seguindo', 'posts']).to_csv('dados/' + usuario + '.csv', sep = ',', index = False)

In [6]:
coleta_dados('jogadores_insta')